In [2]:
import os
import numpy as np
import matplotlib.pyplot as plt
import sys 
import mdtraj

sys.path.insert(0, '/home/wwj/Repo/projects/torchdiffeq/')
sys.path.insert(0, '../')

import torch
from torch.optim import Adam
from torchmd.potentials import ExcludedVolume
from torchmd.sovlers import odeint_adjoint, odeint
from nglview import show_ase, show_file, show_mdtraj

from ase import Atoms

from torchmd.system import GNNPotentials,PairPot,System, Stack
from torchmd.md import Simulations


_ColormakerRegistry()

In [16]:

size = 1
L = 19.73 / size
device = 3

In [ ]:
# create a small water box 


In [4]:
from nff.train import get_model

params = {
    'n_atom_basis': 128,
    'n_filters': 129,
    'n_gaussians': 128,
    'n_convolutions': 32,
    'cutoff': 5.0,
    'trainable_gauss': False
}


# Define prior potential 
lj_params = {'epsilon': 0.005, 
             'sigma': 3.0, 
             'power': 12}


In [5]:
from ase.lattice.cubic import FaceCenteredCubic
from ase import units

atoms = FaceCenteredCubic(directions=[[1, 0, 0], 
                                    [0, 1, 0], [0, 0, 1]],
                          symbol='H',
                          size=(size, size, size),
                          latticeconstant= L,
                          pbc=True)


In [6]:
system.traj

NameError: name 'system' is not defined

In [7]:
system = System(atoms, device=3)
system.set_temperature(298.0)


pair = PairPot(ExcludedVolume, lj_params,
                cell=torch.Tensor(system.get_cell_len()), 
                device=device,
                cutoff=8.0,
                ).to(device)

model = get_model(params)
GNN = GNNPotentials(model, system.get_batch(), system.get_cell_len(), cutoff=5.0, device=system.device)

model = Stack({'gnn': GNN, 'pair': pair})

from torchmd.md import NoseHooverChain 

f_x = NoseHooverChain(model, 
        system.get_masses(), 
        Q=50.0, 
        T=298.0 * units.kB,
        num_chains=5, 
        device=device,
        adjoint=True).to(device)

TypeError: __init__() got an unexpected keyword argument 'device'

In [15]:
system.__

"System(symbols='H256', pbc=True, cell=[19.73, 19.73, 19.73], momenta=...)"

In [13]:
TypeError

torchmd.system.System

In [102]:
system.get_cell_len()

array([19.73, 19.73, 19.73])

In [103]:
system.get_batch()

{'nxyz': tensor([[ 1.0000,  0.0000,  0.0000,  0.0000],
         [ 1.0000,  2.4662,  2.4662,  0.0000],
         [ 1.0000,  2.4662,  0.0000,  2.4662],
         ...,
         [ 1.0000, 17.2638, 17.2638, 14.7975],
         [ 1.0000, 17.2638, 14.7975, 17.2638],
         [ 1.0000, 14.7975, 17.2638, 17.2638]]),
 'num_atoms': tensor([256]),
 'energy': 0.0}

In [108]:
units.kB * 298

0.025679644404907293

In [104]:
sim = Simulations(system, f_x)

In [105]:
trajs = sim.simulate(steps=20, frequency=5)

tensor([0.0000, 0.0982, 0.1965, 0.2947, 0.3929], device='cuda:3')


In [106]:
system.traj[-1][0][0]

array([0.12341952, 0.13180196, 0.03859412], dtype=float32)

In [17]:
system.traj[0][0][0]

array([-0.09430567,  0.0977138 ,  0.04812827])

In [18]:
system.initial_conditions()[-1]

tensor([nan, nan, nan, nan, nan], device='cuda:3')

In [64]:
pair(trajs[1][1])

tensor([[0.0037],
        [0.0041],
        [0.0036],
        ...,
        [0.0041],
        [0.0030],
        [0.0032]], device='cuda:3', grad_fn=<MulBackward0>)

In [86]:
E = GNN(trajs[1][4])

In [87]:
compute_grad(E, trajs[1][4])

RuntimeError: One of the differentiated Tensors appears to not have been used in the graph. Set allow_unused=True if this is the desired behavior.

In [85]:
trajs[1][1]

tensor([[-1.4299e-02, -2.4672e-02, -3.1856e-02],
        [ 2.4149e+00,  2.4269e+00, -9.3615e-02],
        [ 2.4010e+00, -6.3049e-02,  2.3937e+00],
        [-6.4501e-02,  2.4274e+00,  2.4101e+00],
        [ 4.9474e+00, -4.8080e-02, -8.4523e-02],
        [ 7.3733e+00,  2.4034e+00, -1.3170e-01],
        [ 7.3890e+00, -1.4063e-01,  2.3808e+00],
        [ 4.9003e+00,  2.3764e+00,  2.3519e+00],
        [ 9.8829e+00, -8.9019e-02, -6.6553e-02],
        [ 1.2372e+01,  2.4087e+00, -9.8711e-02],
        [ 1.2329e+01, -1.4503e-01,  2.4261e+00],
        [ 9.9005e+00,  2.3270e+00,  2.3176e+00],
        [ 1.4811e+01, -3.9825e-02, -6.8312e-02],
        [ 1.7292e+01,  2.4485e+00, -5.2785e-02],
        [ 1.7303e+01, -6.1432e-02,  2.4557e+00],
        [ 1.4888e+01,  2.3920e+00,  2.3592e+00],
        [-6.6838e-02,  4.8932e+00, -5.8865e-02],
        [ 2.3889e+00,  7.4035e+00, -1.1835e-01],
        [ 2.3308e+00,  4.9015e+00,  2.3652e+00],
        [-1.5541e-01,  7.3598e+00,  2.4178e+00],
        [ 4.9049e+00

In [1]:
system.get_velocities()

NameError: name 'system' is not defined

In [168]:
# 
Observable(system)

Observable()

In [169]:
obs = rdf(system , 10, 5.0)